In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
os.chdir('/home/joe/testbed/DeepSurvivalMachines')

In [6]:
from dsm import datasets
x, t, e = datasets.load_dataset('PBC', sequential = True)

In [7]:
import numpy as np
import seaborn as sns
import pandas as pd

In [8]:
import torch
import tqdm
import torch.nn as nn
import torch.nn.functional as F

# reusing compression code via gumbel

In [9]:
import utils

In [10]:
bs = 128 # batch size

In [11]:
torch.cuda.is_available()

True

In [12]:
(x_train, t_train, e_train,
            x_val, t_val, e_val) = utils.prepocess_training_data_from_lists(x, t, e, vsize=0.2, val_data=None, random_state=563)

In [13]:
print(x_train.shape, t_train.shape, e_train.shape)

torch.Size([250, 16, 25]) torch.Size([250, 16, 1]) torch.Size([250, 16, 1])


In [14]:
scm_model = utils.SurvivalClusteringMachine(x_train.shape[2], hidden=10, 
                k=4, lstm_layers=1, layers=None, dist='Weibull',
               temp=1000., discount=1.0, optimizer='Adam',
               risks=1)

In [15]:
xrep = scm_model.represent(x_train)

In [16]:
xrep.shape

torch.Size([250, 20])

In [17]:
#shape, scale, logits = model.forward(x, risk)
shape, scale, logits = scm_model.forward(x_train, '1')

In [18]:
print(shape.shape, scale.shape, logits.shape)

torch.Size([250, 4]) torch.Size([250, 4]) torch.Size([250, 4])


In [19]:
utils._conditional_weibull_loss(scm_model, x_train, t_train, e_train)

torch.Size([250, 4]) torch.Size([250, 4, 16, 250])


RuntimeError: The size of tensor a (4) must match the size of tensor b (250) at non-singleton dimension 3

In [38]:
k

tensor([-0.9273, -0.7244, -0.6787, -0.7432, -0.7416, -0.8662, -0.8068, -0.9400,
        -0.7783, -0.7434, -0.6815, -0.7831, -0.7866, -0.8228, -0.7006, -0.8271,
        -0.7496, -0.6798, -0.7254, -0.7358, -0.7330, -0.9491, -0.8368, -0.7053,
        -0.6761, -0.8167, -0.8656, -0.8977, -0.8484, -0.7892, -0.7107, -0.7391,
        -0.7416, -0.7175, -0.7625, -0.7754, -0.8630, -0.7329, -0.8907, -0.8116,
        -0.7103, -0.6198, -0.6961, -0.7975, -0.7208, -0.8330, -0.7480, -0.6859,
        -0.7438, -0.6829, -0.8024, -0.8919, -0.6860, -0.7387, -0.8475, -0.8134,
        -0.8906, -0.7604, -0.9296, -0.8239, -0.7637, -0.9223, -0.8198, -0.6393,
        -0.8780, -0.7182, -0.7673, -0.7144, -0.8102, -0.7479, -0.6816, -0.6999,
        -0.6973, -0.7636, -0.8127, -0.7990, -0.8057, -0.7330, -0.7217, -0.7102,
        -0.8105, -0.7630, -0.7694, -0.7383, -0.7237, -0.7911, -0.8491, -0.9511,
        -0.6457, -0.8812, -0.7502, -0.8524, -0.7161, -0.7128, -0.8915, -0.7420,
        -0.7260, -0.7838, -0.9144, -0.76

In [28]:
t = t_train

lossf = []
losss = []

k_ = shape
b_ = scale
for g in range(scm_model.k):

    k = k_[:, g]
    b = b_[:, g]

    s = - (torch.pow(torch.exp(b)*t, torch.exp(k)))
    f = k + b + ((torch.exp(k)-1)*(b+torch.log(t)))
    f = f + s

    lossf.append(f)
    losss.append(s)

losss = torch.stack(losss, dim=1)
lossf = torch.stack(lossf, dim=1)

lossg = nn.Softmax(dim=1)(logits)

In [25]:
losss.shape

torch.Size([250, 4, 16, 250])

In [29]:
lossg.shape

torch.Size([250, 4])

In [2]:
def _conditional_weibull_loss(model, x, t, e, elbo=True, risk='1'):

  alpha = model.discount
  shape, scale, logits = model.forward(x, risk)

  k_ = shape
  b_ = scale

  lossf = []
  losss = []

  for g in range(model.k):

    k = k_[:, g]
    b = b_[:, g]

    s = - (torch.pow(torch.exp(b)*t, torch.exp(k)))
    f = k + b + ((torch.exp(k)-1)*(b+torch.log(t)))
    f = f + s

    lossf.append(f)
    losss.append(s)

  losss = torch.stack(losss, dim=1)
  lossf = torch.stack(lossf, dim=1)

  if elbo:

    lossg = nn.Softmax(dim=1)(logits)
    losss = lossg*losss
    lossf = lossg*lossf
    losss = losss.sum(dim=1)
    lossf = lossf.sum(dim=1)

  else:

    lossg = nn.LogSoftmax(dim=1)(logits)
    losss = lossg + losss
    lossf = lossg + lossf
    losss = torch.logsumexp(losss, dim=1)
    lossf = torch.logsumexp(lossf, dim=1)

  uncens = np.where(e.cpu().data.numpy() == int(risk))[0]
  cens = np.where(e.cpu().data.numpy() != int(risk))[0]
  ll = lossf[uncens].sum() + alpha*losss[cens].sum()

  return -ll/float(len(uncens)+len(cens))

In [3]:
utils.train_scm(scm_model, x_train, t_train, e_train)

NameError: name 'utils' is not defined

In [31]:
scm_model.gate['1'](xrep).shape

torch.Size([250, 4])

In [32]:
scm_model.shapeg['1'](xrep).shape

torch.Size([250, 4])